In [1]:
from statsbombpy import sb
import os
import pandas as pd
import sqlite3
import warnings

warnings.filterwarnings("ignore")

connect = sqlite3.connect("../../raw-database.db")

email = "nathan.talbot@etu.uca.fr"
password = os.environ["mdp_statsbomb"]
creds = {"user" : email, "passwd" : password}

liste_saison = ["2023/2024", "2022/2023", "2021/2022", "2020/2021"]

data = pd.concat([sb.competition_events(country = "France", division = "Ligue 2", season = saison, gender = "male", creds = creds)
                for saison in liste_saison])

ModuleNotFoundError: No module named 'statsbombpy'

In [4]:
colonnes = ["shot_type", "shot_outcome", "type", "match_id", "period", "possession", "location", "team", "pass_cross", "pass_type", "index",
            "pass_end_location", "minute", "player", "shot_end_location", "pass_body_part"]

df = data[colonnes].reset_index().rename({"index" : "index_event"}, axis = 1)

df_loc = df.pop("location").dropna()
df_loc = pd.DataFrame(df_loc.tolist(), index = df_loc.index, columns = ["x_loc", "y_loc", "z_loc"])

df_pass_loc = df.pop("pass_end_location").dropna()
df_pass_loc = pd.DataFrame(df_pass_loc.tolist(), index = df_pass_loc.index, columns = ["x_pass", "y_pass"])

df_shot_loc = df.pop("shot_end_location").dropna()
df_shot_loc = pd.DataFrame(df_shot_loc.tolist(), index = df_shot_loc.index, columns = ["x_shot", "y_shot", "z_shot"])

df = pd.concat([df, df_loc, df_pass_loc, df_shot_loc], axis = 1)

df.to_sql(name = "raw_data_heatmap", con = connect, if_exists = "replace", index = False)

connect.close()